https://docs.google.com/spreadsheets/d/10QCryRqYBlS-kE_ExHaGaSZU2JfS88BGcBss-KYr0Wk/edit#gid=0

In [60]:
from utilities.google_client import google_sheet_API
from utilities.athena_client import athena_API
import multiprocessing as mp

import pandas as pd
from configparser import ConfigParser
import os
import pyathena
import time

In [64]:
from botocore.exceptions import ClientError
try:
    athenaAPI.get_pandas_df('Descrive')
except Exception as e:
    print(e)

Failed to execute query.
Traceback (most recent call last):
  File "/Users/emanuele.bonura/.local/share/virtualenvs/ams-alert-monitoring-0ROOZaQw/lib/python3.7/site-packages/pyathena/common.py", line 206, in _execute
    **request).get('QueryExecutionId', None)
  File "/Users/emanuele.bonura/.local/share/virtualenvs/ams-alert-monitoring-0ROOZaQw/lib/python3.7/site-packages/pyathena/util.py", line 214, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/Users/emanuele.bonura/.local/share/virtualenvs/ams-alert-monitoring-0ROOZaQw/lib/python3.7/site-packages/tenacity/__init__.py", line 391, in call
    do = self.iter(retry_state=retry_state)
  File "/Users/emanuele.bonura/.local/share/virtualenvs/ams-alert-monitoring-0ROOZaQw/lib/python3.7/site-packages/tenacity/__init__.py", line 338, in iter
    return fut.result()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File

Execution failed on sql: Descrive
An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:1: mismatched input 'Descrive' expecting {'(', 'SELECT', 'DESC', 'USING', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'INSERT', 'DELETE', 'DESCRIBE', 'GRANT', 'REVOKE', 'EXPLAIN', 'SHOW', 'USE', 'DROP', 'ALTER', 'SET', 'RESET', 'START', 'COMMIT', 'ROLLBACK', 'CALL', 'PREPARE', 'DEALLOCATE', 'EXECUTE'}
unable to rollback


In [40]:
# Initialise the 2 APIs 
gsheetAPI = google_sheet_API()
athenaAPI = athena_API()

In [41]:
# Retrieve the sheet and add the queries
gsheet_df = gsheetAPI.retrieve_sheet_as_df()
gsheet_df_queries = athenaAPI.add_row_queries(gsheet_df)

# Get all the queries into one array
queries = gsheet_df_queries['tw_query'].append(gsheet_df_queries['lw_query'], ignore_index=True)

In [42]:
print(f'Beginning multiprocessing at {time.asctime()}, running {len(queries)} queries on {mp.cpu_count()} threads')

Beginning multiprocessing at Thu Apr  2 12:26:19 2020, running 10 queries


In [43]:
# Multiprocessing to run the queries simultaneously
# It has to stay in the main function
pool = mp.Pool(int(mp.cpu_count()/2))
results = [pool.apply(athenaAPI.get_pandas_df, args=[query]) for query in queries]

In [44]:
# Update the df with the count
# This step is needed in case the results are not returned in the correct order
results_list = [v.values[0][0] for v in results]
results_dict = dict(zip(queries, results_list))
gsheet_df_queries['tw_count'] = gsheet_df_queries['tw_query'].map(results_dict)
gsheet_df_queries['lw_count'] = gsheet_df_queries['lw_query'].map(results_dict)

In [45]:
#gsheet_df_queries = pd.DataFrame({'tw_count':  [112160,   81407, 1240195,     143,    1229], 'lw_count': [132589,    98960,  1395811,      156,     1138]})

In [46]:
gsheet_df_queries

,name,domain,perc_increase,account_id,site_id,url,and_condition,note,attack_category,owner,tw_count,lw_count,perc_increase_float,tw_query,lw_query
0,first test attack alerting,www.realtor.com,5%,20f61841-e844-4532-8c08-dcd517daabb8,f44b7a05-3268-451b-ae56-79dddd9199c9,realestat,AND action is NULL,test to check if this works,,Emanuele,123958,145477,0.05,SELECT COUNT(*)\n FROM bon_...,SELECT COUNT(*)\n FROM bon_...
1,first test attack alerting,www.realtor.com,7%,20f61841-e844-4532-8c08-dcd517daabb8,f44b7a05-3268-451b-ae56-79dddd9199c9,realestateandhomes-detai,AND action is NULL,test to check if this works,,Emanuele,89709,107440,0.07,SELECT COUNT(*)\n FROM bon_...,SELECT COUNT(*)\n FROM bon_...
2,first test attack alerting,www.realtor.com,10%,20f61841-e844-4532-8c08-dcd517daabb8,f44b7a05-3268-451b-ae56-79dddd9199c9,,AND action is NULL,test to check if this works,,Emanuele,1365771,1532775,0.10,SELECT COUNT(*)\n FROM bon_...,SELECT COUNT(*)\n FROM bon_...
3,first test attack alerting,www.realtor.com,15%,20f61841-e844-4532-8c08-dcd517daabb8,f44b7a05-3268-451b-ae56-79dddd9199c9,sell,AND action is NULL,test to check if this works,,Emanuele,160,181,0.15,SELECT COUNT(*)\n FROM bon_...,SELECT COUNT(*)\n FROM bon_...
4,first test attack alerting,mercury.worldremit.com,5%,2e8d6f13-c651-46ca-bfe5-2293e4586ede,32261b05-3a41-45df-ae4c-199945da4c96,/auth/login,AND action is NULL,test to check if this works,,Atul,1354,1189,0.05,SELECT COUNT(*)\n FROM bon_...,SELECT COUNT(*)\n FROM bon_...


In [47]:
# Update sheet
for row in gsheet_df_queries[['tw_count', 'lw_count']].iterrows():
    cell_address = f'K{row[0]+2}'
    cell_value = [int(row[1]['tw_count']), int(row[1]['lw_count'])]
    result = gsheetAPI.update_sheet(cell_address, cell_value)


{'spreadsheetId': '10QCryRqYBlS-kE_ExHaGaSZU2JfS88BGcBss-KYr0Wk', 'updatedRange': 'queries!K2:L2', 'updatedRows': 1, 'updatedColumns': 2, 'updatedCells': 2}
{'spreadsheetId': '10QCryRqYBlS-kE_ExHaGaSZU2JfS88BGcBss-KYr0Wk', 'updatedRange': 'queries!K3:L3', 'updatedRows': 1, 'updatedColumns': 2, 'updatedCells': 2}
{'spreadsheetId': '10QCryRqYBlS-kE_ExHaGaSZU2JfS88BGcBss-KYr0Wk', 'updatedRange': 'queries!K4:L4', 'updatedRows': 1, 'updatedColumns': 2, 'updatedCells': 2}
{'spreadsheetId': '10QCryRqYBlS-kE_ExHaGaSZU2JfS88BGcBss-KYr0Wk', 'updatedRange': 'queries!K5:L5', 'updatedRows': 1, 'updatedColumns': 2, 'updatedCells': 2}
{'spreadsheetId': '10QCryRqYBlS-kE_ExHaGaSZU2JfS88BGcBss-KYr0Wk', 'updatedRange': 'queries!K6:L6', 'updatedRows': 1, 'updatedColumns': 2, 'updatedCells': 2}
